In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ctrace
from tqdm import tqdm
from ctrace.simulation import *
from ctrace.utils import *

from ctrace.dataset import load_graph, load_sir_path
import concurrent.futures

In [3]:
with open(ctrace.PROJECT_ROOT /"output"/"run_K2VWk" / "time_trials_final.csv", "r") as csv:
    df = pd.read_csv(csv)
df.head()

,Unnamed: 0,Unnamed: 0.1,G,p,budget,from_cache,method,trial_id,min_exposed_value,mip_value,...,I_size,v1_size,v2_size,num_cross_edges,duration,updated_MIP,v1_sum_values,min_exposed_mean,min_exposed_stddev,updated_min_exposed_value
0,0,0,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,1,"(118.8, 12.517987058628878)",90.850904,...,1217,6233,11135,16931,0.883242,752.813311,661.962407,118.8,12.517987,780.762407
1,1,1,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,"(123.6, 9.50263121456368)",90.716560,...,1217,6233,11135,16931,0.885397,752.678967,661.962407,123.6,9.502631,785.562407
2,2,2,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,"(95.2, 14.078352176302452)",69.442667,...,1217,6233,11135,16931,1.148150,731.405074,661.962407,95.2,14.078352,757.162407
3,3,3,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,"(96.6, 11.567194992737004)",69.491244,...,1217,6233,11135,16931,1.222075,731.453652,661.962407,96.6,11.567195,758.562407
4,4,4,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,"(93.2, 11.966620241321273)",71.084567,...,1217,6233,11135,16931,0.032429,733.046974,661.962407,93.2,11.966620,755.162407


In [4]:
list(df)

['Unnamed: 0',
 'Unnamed: 0.1',
 'G',
 'p',
 'budget',
 'from_cache',
 'method',
 'trial_id',
 'min_exposed_value',
 'mip_value',
 'greedy_intersection',
 'maxD',
 'I_size',
 'v1_size',
 'v2_size',
 'num_cross_edges',
 'duration',
 'updated_MIP',
 'v1_sum_values',
 'min_exposed_mean',
 'min_exposed_stddev',
 'updated_min_exposed_value']

In [8]:
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [11]:
df = df.drop(['min_exposed_value'], axis=1)

In [38]:
df.head()

,G,p,budget,from_cache,method,trial_id,mip_value,greedy_intersection,maxD,I_size,v1_size,v2_size,num_cross_edges,duration,updated_MIP,v1_sum_values,min_exposed_mean,min_exposed_stddev,updated_min_exposed_value
0,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,1,90.850904,25,28,1217,6233,11135,16931,0.883242,752.813311,661.962407,118.8,12.517987,780.762407
1,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,90.716560,20,28,1217,6233,11135,16931,0.885397,752.678967,661.962407,123.6,9.502631,785.562407
2,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,69.442667,256,28,1217,6233,11135,16931,1.148150,731.405074,661.962407,95.2,14.078352,757.162407
3,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,69.491244,252,28,1217,6233,11135,16931,1.222075,731.453652,661.962407,96.6,11.567195,758.562407
4,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,71.084567,-1,28,1217,6233,11135,16931,0.032429,733.046974,661.962407,93.2,11.966620,755.162407


In [39]:
len(df)

14000

In [31]:
methods = {x: df[df["method"] == x].copy() for x in ["random", "dependent", "greedy_weighted", "mip_gurobi"]}
methods["mip_gurobi"].head()

,G,p,budget,from_cache,method,trial_id,mip_value,greedy_intersection,maxD,I_size,v1_size,v2_size,num_cross_edges,duration,updated_MIP,v1_sum_values,min_exposed_mean,min_exposed_stddev,updated_min_exposed_value
6,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,68.296894,273,28,1217,6233,11135,16931,1.288606,730.259302,661.962407,94.0,14.035669,755.962407
8,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,1,68.296894,273,28,1217,6233,11135,16931,1.331758,730.259302,661.962407,96.0,1.000000,757.962407
28,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,1,255.187305,207,31,5772,26012,16285,42903,7.722165,5094.834993,4839.647687,494.8,56.984208,5334.447687
35,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,255.187305,207,31,5772,26012,16285,42903,6.833029,5094.834993,4839.647687,477.8,15.254508,5317.447687
37,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,1,452.765661,250,59,2359,25664,32967,145552,27.815548,4186.409030,3733.643370,1230.6,49.303144,4964.243370


In [34]:
optimal_set = methods["mip_gurobi"][["G", "p", "budget", "from_cache", "trial_id", "updated_MIP"]]

In [36]:
filtered_optimal = optimal_set[optimal_set["trial_id"] == 0]

In [37]:
filtered_optimal

,G,p,budget,from_cache,trial_id,updated_MIP
6,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,0,730.259302
35,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,0,5094.834993
44,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,0,4186.409030
46,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,0,6079.534783
48,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,0,5288.797413
...,...,...,...,...,...,...
13978,montgomery,0.078,1000,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,0,2130.513870
13986,montgomery,0.078,1000,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,0,1769.977871
13989,montgomery,0.078,1000,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,0,3483.896082
13997,montgomery,0.078,1000,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,0,5111.010541


In [45]:
df_opt = df.merge(filtered_optimal, on=["G", "p", "budget", "from_cache"])

In [48]:
df_opt = df_opt.rename(columns={"updated_MIP_y": "optimal_MIP_value"})

In [49]:
df_opt["mip_ratio"] = df_opt["updated_MIP_x"] / df_opt["optimal_MIP_value"]

In [51]:
df_opt.head()

,G,p,budget,from_cache,method,trial_id_x,mip_value,greedy_intersection,maxD,I_size,...,num_cross_edges,duration,updated_MIP_x,v1_sum_values,min_exposed_mean,min_exposed_stddev,updated_min_exposed_value,trial_id_y,optimal_MIP_value,mip_ratio
0,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,1,90.850904,25,28,1217,...,16931,0.883242,752.813311,661.962407,118.8,12.517987,780.762407,0,730.259302,1.030885
1,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,90.716560,20,28,1217,...,16931,0.885397,752.678967,661.962407,123.6,9.502631,785.562407,0,730.259302,1.030701
2,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,69.442667,256,28,1217,...,16931,1.148150,731.405074,661.962407,95.2,14.078352,757.162407,0,730.259302,1.001569
3,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,69.491244,252,28,1217,...,16931,1.222075,731.453652,661.962407,96.6,11.567195,758.562407,0,730.259302,1.001636
4,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,71.084567,-1,28,1217,...,16931,0.032429,733.046974,661.962407,93.2,11.966620,755.162407,0,730.259302,1.003817


In [52]:
df_opt.sort_values(by=["v1_size"])

,G,p,budget,from_cache,method,trial_id_x,mip_value,greedy_intersection,maxD,I_size,...,num_cross_edges,duration,updated_MIP_x,v1_sum_values,min_exposed_mean,min_exposed_stddev,updated_min_exposed_value,trial_id_y,optimal_MIP_value,mip_ratio
1524,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,28.727917,-1,23,529,...,8273,0.025445,304.862589,276.134672,30.2,3.962323,306.334672,0,303.693238,1.003850
5526,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,1,22.628561,-1,23,529,...,8273,0.016595,298.763233,276.134672,23.0,4.242641,299.134672,0,297.755343,1.003385
5525,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,1,21.620672,509,23,529,...,8273,0.388629,297.755343,276.134672,27.4,4.335897,303.534672,0,297.755343,1.000000
5524,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,22.628561,-1,23,529,...,8273,0.016042,298.763233,276.134672,23.8,1.643168,299.934672,0,297.755343,1.003385
5523,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,1,21.737258,501,23,529,...,8273,0.389772,297.871930,276.134672,23.4,6.877500,299.534672,0,297.755343,1.000392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,373.962885,182,29,4830,...,78816,14.311749,6269.828710,5895.865825,886.8,23.763417,6782.665825,0,6269.828710,1.000000
151,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,1,373.962885,182,29,4830,...,78816,16.694823,6269.828710,5895.865825,874.6,19.654516,6770.465825,0,6269.828710,1.000000
4149,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,372.130173,226,29,4830,...,78816,7.278213,6267.995997,5895.865825,882.8,40.733279,6778.665825,0,6254.578511,1.002145
8150,montgomery,0.078,800,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,1,345.484920,406,29,4830,...,78816,13.935759,6241.350745,5895.865825,825.6,56.411878,6721.465825,0,6241.350745,1.000000


In [55]:
df_opt.to_csv("with_optimal.csv")

In [20]:
df_stats = df[['G','p', 'budget', 'from_cache', 'trial_id', 'maxD', 'I_size', 'v1_size', 'v2_size', 'num_cross_edges']]

In [26]:
methods["mip_gurobi"]["updated_mip_gurobi"] = methods["mip_gurobi"]["updated_MIP"]

In [28]:
methods["mip_gurobi"].head()
processed = {}

In [29]:
processed["mip_gurobi"] = methods["mip_gurobi"][['G','p', 'budget', 'from_cache', 'trial_id', 'maxD', 'I_size', 'v1_size', 'v2_size', 'num_cross_edges', "updated_mip_gurobi"]]

In [30]:
processed["mip_gurobi"].head()

,G,p,budget,from_cache,trial_id,maxD,I_size,v1_size,v2_size,num_cross_edges,updated_mip_gurobi
6,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,0,28,1217,6233,11135,16931,730.259302
8,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,1,28,1217,6233,11135,16931,730.259302
28,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,1,31,5772,26012,16285,42903,5094.834993
35,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,0,31,5772,26012,16285,42903,5094.834993
37,montgomery,0.078,400,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,1,59,2359,25664,32967,145552,4186.409030
